#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/competitions/precio-portatiles-the-bridge/overview

## Librerias

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from Funciones import *

## Data

- Variable a predecir: Price_euros

- Variables predictoras: por defenir

- Variables a desechar: laptop_ID, 

In [2]:
df_test = pd.read_csv('../data/test.csv', index_col=0)
# df_test

In [3]:
df_train = pd.read_csv('../data/train.csv', index_col=0)
# df_train

In [4]:
sample = pd.read_csv('../data/sample_submission.csv')
# sample

## Data exploration

In [5]:
# df_train['TypeName'].nunique()

In [6]:
# df_train['TypeName'].value_counts()

In [7]:
# df_train['OpSys'].value_counts()

## Data processing

## Test

In [8]:
# Para este primer modelo voy a limpiar algunas columnas y cambiar su dtype

#Primero ram
df_test['Ram'] = df_test['Ram'].str.replace('GB', '').astype(int)

# Ahora voy a transformar Weight
df_test['Weight'] = df_test['Weight'].str.replace('kg', '').astype(float)

# Voy a renombar  Weight y le pondré Weigh/kg para que sea más descriptiva y se entienda que los valores numericos hacen referencia a su peso en kg

df_test = df_test.rename(columns={'Weight': 'Weight/kg'})

In [9]:
# Me haré un segundo DF donde desecho las primeras dos columnas porque no las necesito para la predicción

df2_test = df_test.drop(['laptop_ID'], axis=1)

# Voy a ver si puedo sacarle provecho al typeName
# df2_t = df2_t.drop(['TypeName'], axis=1)

In [10]:
# Función que agrupa OS
def group_os(name):
    if 'Windows' in name:
        return 'Windows'
    elif re.search(r'mac', name, re.IGNORECASE):
        return 'Mac'
    elif name in ['Linux', 'Chrome OS', 'Android']:
        return 'Other OS'
    else:
        return 'No OS/Other'

In [11]:
# Extraigo los pixeles
def extraer_pixeles(resolucion):
    # Busca dos números separados por 'x' o 'X'
    match = re.search(r'(\d+)x(\d+)', resolucion, re.IGNORECASE)
    if match:
        ancho, alto = map(int, match.groups())
        return ancho * alto
    else:
        return None  # O un valor por defecto, como 0 o -1


In [12]:
# Función extrae los screen featuring
def screen_featuring(row):
    ips = 'IPS' in row
    touchscreen = 'Touchscreen' in row
    return pd.Series([ips, touchscreen])

In [13]:
def extraer_gb(memoria):
    total_gb = 0

    # Buscar GB
    matches_gb = re.findall(r'(\d+)GB', memoria)
    for match in matches_gb:
        total_gb += int(match)

    # Buscar TB y convertir a GB (1 TB = 1024 GB)
    matches_tb = re.findall(r'(\d+)TB', memoria)
    for match in matches_tb:
        total_gb += int(match) * 1024

    return total_gb if total_gb > 0 else 0  # Devuelve 0 si no se encuentra nada


In [14]:
def codificar_tipo_memoria(memoria):
    if 'SSD' in memoria:
        return 1
    elif 'HDD' in memoria:
        return 2
    elif 'Hybrid' in memoria:
        return 3
    elif 'Flash' in memoria:
        return 4
    else:
        return 0  # O algún otro código para tipos no listados

In [15]:
#  Agrupar por marcas
def asignar_marca_gpu(gpu):
    if 'Intel' in gpu:
        return 'Intel'
    elif 'Nvidia' in gpu:
        return 'Nvidia'
    elif 'AMD' in gpu:
        return 'AMD'
    else:
        return 'Otro'

In [16]:
def clasificar_cpu(cpu):
    if 'i7' in cpu or 'i9' in cpu or 'Ryzen 7' in cpu or 'Ryzen 9' in cpu:
        return 'Alta'
    # elif 'i5' in cpu or 'Ryzen 5' in cpu:
    #     return 'Media'
    else:
        return 'Baja'

In [17]:
def extraer_ghz(cpu):
    match = re.search(r'(\d+\.?\d*)GHz', cpu)
    return float(match.group(1)) if match else 0

In [18]:
# Agrupo varias variables
df2_test['OpSys_Grouped'] = df2_test['OpSys'].apply(group_os)
df2_test['Company_Grouped'] = df2_test['Company'].apply(agrupar_marcas)

# Extraer los píxeles
df2_test['Pixeles_Total'] = df2_test['ScreenResolution'].apply(extraer_pixeles)

# Extraer características de la pantalla
df2_test[['IPS', 'Touchscreen']] = df2_test['ScreenResolution'].apply(screen_featuring)

# Memoria
df2_test['Total_Memory_GB'] = df2_test['Memory'].apply(extraer_gb)
df2_test['Memory_Type_Code'] = df2_test['Memory'].apply(codificar_tipo_memoria)

# Aplico la función asignar brand a GPU
df2_test['GPU_Marca'] = df2_test['Gpu'].apply(asignar_marca_gpu)

# Clasificamos los CPU
df2_test['CPU_Gama'] = df2_test['Cpu'].apply(clasificar_cpu)

# Aplicar la función para extraer la velocidad de GHz
df2_test['CPU_GHz'] = df2_test['Cpu'].apply(extraer_ghz)

In [19]:
df2_test

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,...,OpSys_Grouped,Company_Grouped,Pixeles_Total,IPS,Touchscreen,Total_Memory_GB,Memory_Type_Code,GPU_Marca,CPU_Gama,CPU_GHz
id,,,,,,,,,,,,,,,,,,,,,
181,HP,Spectre x360,Ultrabook,13.3,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7500U 2.7GHz,16,512GB SSD,Intel HD Graphics 620,Windows 10,...,Windows,HP,8294400,True,False,512,1,Intel,Alta,2.7
708,Acer,Aspire 5,Notebook,15.6,1366x768,AMD A12-Series 9720P 2.7GHz,8,256GB SSD,AMD Radeon RX 540,Windows 10,...,Windows,Acer,1049088,False,False,256,1,AMD,Baja,2.7
862,Acer,Aspire ES1-572,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4,500GB HDD,Intel HD Graphics 520,Linux,...,Other OS,Acer,1049088,False,False,500,2,Intel,Baja,2.0
1064,HP,EliteBook 1040,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8,256GB SSD,Intel HD Graphics 520,Windows 7,...,Windows,HP,2073600,False,False,256,1,Intel,Baja,2.3
702,HP,ENVY -,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,Windows 10,...,Windows,HP,2073600,True,False,256,1,Intel,Baja,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,Lenovo,Legion Y520-15IKBN,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8,256GB SSD,Nvidia GeForce GTX 1050M,No OS,...,No OS/Other,Lenovo,2073600,True,False,256,1,Nvidia,Alta,2.8
524,Lenovo,IdeaPad Y700-15ISK,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16,512GB SSD,Nvidia GeForce GTX 960,Windows 10,...,Windows,Lenovo,2073600,True,False,512,1,Nvidia,Alta,2.6
1015,HP,250 G5,Notebook,15.6,1366x768,Intel Core i5 7200U 2.5GHz,4,500GB HDD,Intel HD Graphics 620,No OS,...,No OS/Other,HP,1049088,False,False,500,2,Intel,Baja,2.5


In [20]:
# Dummies_os
dummies_os_test = pd.get_dummies(df2_test, columns=["Company_Grouped"])

In [21]:
# Dummies_comp
dummies_com_test = pd.get_dummies(df2_test, columns=["Company_Grouped"])

# Creo dummies para CPU_Gama

cpu_gama_dummies = pd.get_dummies(df2_test['CPU_Gama'], prefix='CPU')

# Creo dummies para GPU_Marca
marca_gpu_dummies = pd.get_dummies(df2_test['GPU_Marca'], prefix='GPU')

In [22]:
# Aplicar One-Hot Encoding a la variable TypeName, al ser tan pocos no hizo falta agruparlos primero
df_encoded_test = pd.get_dummies(df2_test, columns=['TypeName'])
df_encoded_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 181 to 1036
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Company                      391 non-null    object 
 1   Product                      391 non-null    object 
 2   Inches                       391 non-null    float64
 3   ScreenResolution             391 non-null    object 
 4   Cpu                          391 non-null    object 
 5   Ram                          391 non-null    int32  
 6   Memory                       391 non-null    object 
 7   Gpu                          391 non-null    object 
 8   OpSys                        391 non-null    object 
 9   Weight/kg                    391 non-null    float64
 10  OpSys_Grouped                391 non-null    object 
 11  Company_Grouped              391 non-null    object 
 12  Pixeles_Total                391 non-null    int64  
 13  IPS                   

In [23]:
# Uno GPU_dummies con encoded
df_encoded_test_2 = pd.concat([df_encoded_test, marca_gpu_dummies], axis=1)
df_encoded_test_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 181 to 1036
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Company                      391 non-null    object 
 1   Product                      391 non-null    object 
 2   Inches                       391 non-null    float64
 3   ScreenResolution             391 non-null    object 
 4   Cpu                          391 non-null    object 
 5   Ram                          391 non-null    int32  
 6   Memory                       391 non-null    object 
 7   Gpu                          391 non-null    object 
 8   OpSys                        391 non-null    object 
 9   Weight/kg                    391 non-null    float64
 10  OpSys_Grouped                391 non-null    object 
 11  Company_Grouped              391 non-null    object 
 12  Pixeles_Total                391 non-null    int64  
 13  IPS                   

In [24]:
# Uno CPU_dummies con Encoded
df_encoded_test_3 = pd.concat([df_encoded_test_2, cpu_gama_dummies], axis=1)
df_encoded_test_3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 181 to 1036
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Company                      391 non-null    object 
 1   Product                      391 non-null    object 
 2   Inches                       391 non-null    float64
 3   ScreenResolution             391 non-null    object 
 4   Cpu                          391 non-null    object 
 5   Ram                          391 non-null    int32  
 6   Memory                       391 non-null    object 
 7   Gpu                          391 non-null    object 
 8   OpSys                        391 non-null    object 
 9   Weight/kg                    391 non-null    float64
 10  OpSys_Grouped                391 non-null    object 
 11  Company_Grouped              391 non-null    object 
 12  Pixeles_Total                391 non-null    int64  
 13  IPS                   

In [25]:
# Uno Dummies_os_company con Encoded
df_encoded_test_4 = pd.concat([df_encoded_test_3, dummies_os_test], axis=1)
df_encoded_test_4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 181 to 1036
Data columns (total 61 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Company                      391 non-null    object 
 1   Product                      391 non-null    object 
 2   Inches                       391 non-null    float64
 3   ScreenResolution             391 non-null    object 
 4   Cpu                          391 non-null    object 
 5   Ram                          391 non-null    int32  
 6   Memory                       391 non-null    object 
 7   Gpu                          391 non-null    object 
 8   OpSys                        391 non-null    object 
 9   Weight/kg                    391 non-null    float64
 10  OpSys_Grouped                391 non-null    object 
 11  Company_Grouped              391 non-null    object 
 12  Pixeles_Total                391 non-null    int64  
 13  IPS                   

In [26]:
# Uno Dummies_os_company con Encoded
df_encoded_test_5 = pd.concat([df_encoded_test_4, dummies_com_test], axis=1)
df_encoded_test_5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 181 to 1036
Data columns (total 90 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Company                      391 non-null    object 
 1   Product                      391 non-null    object 
 2   Inches                       391 non-null    float64
 3   ScreenResolution             391 non-null    object 
 4   Cpu                          391 non-null    object 
 5   Ram                          391 non-null    int32  
 6   Memory                       391 non-null    object 
 7   Gpu                          391 non-null    object 
 8   OpSys                        391 non-null    object 
 9   Weight/kg                    391 non-null    float64
 10  OpSys_Grouped                391 non-null    object 
 11  Company_Grouped              391 non-null    object 
 12  Pixeles_Total                391 non-null    int64  
 13  IPS                   

In [27]:
# # Eliminar las columnas especificadas
df_encoded_test_5.drop(['Company', 'Product', 'Cpu', 'Memory', 'Gpu', 'OpSys'], axis=1, inplace=True)
# Eliminar las columnas especificadas
df_encoded_test_5.drop(['OpSys_Grouped', 'Company_Grouped'], axis=1, inplace=True)
df_encoded_test_5.drop(['GPU_Marca', 'CPU_Gama', 'ScreenResolution'], axis=1, inplace=True)

In [28]:
df_encoded_test_5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 181 to 1036
Data columns (total 59 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Inches                       391 non-null    float64
 1   Ram                          391 non-null    int32  
 2   Weight/kg                    391 non-null    float64
 3   Pixeles_Total                391 non-null    int64  
 4   IPS                          391 non-null    bool   
 5   Touchscreen                  391 non-null    bool   
 6   Total_Memory_GB              391 non-null    int64  
 7   Memory_Type_Code             391 non-null    int64  
 8   CPU_GHz                      391 non-null    float64
 9   TypeName_2 in 1 Convertible  391 non-null    bool   
 10  TypeName_Gaming              391 non-null    bool   
 11  TypeName_Netbook             391 non-null    bool   
 12  TypeName_Notebook            391 non-null    bool   
 13  TypeName_Ultrabook    

## Train

In [29]:
# Para este primer modelo voy a limpiar algunas columnas y cambiar su dtype

#Primero ram
1df_train['Ram'] = df_train['Ram'].str.replace('GB', '').astype(int)

# Ahora voy a transformar Weight
df_train['Weight'] = df_train['Weight'].str.replace('kg', '').astype(float)

# Voy a renombar  Weight y le pondré Weigh/kg para que sea más descriptiva y se entienda que los valores numericos hacen referencia a su peso en kg

df_train = df_train.rename(columns={'Weight': 'Weight/kg'})

SyntaxError: invalid decimal literal (3672629820.py, line 4)

In [ ]:
# Me haré un segundo DF donde desecho las primeras dos columnas porque no las necesito para la predicción

df2_train = df_train.drop(['laptop_ID'], axis=1)

# Voy a ver si puedo sacarle provecho al typeName
# df2 = df2.drop(['TypeName'], axis=1)

In [ ]:
# Agrupo varias variables
df2_train['OpSys_Grouped'] = df2_train['OpSys'].apply(group_os)
df2_train['Company_Grouped'] = df2_train['Company'].apply(agrupar_marcas)

# Extraigo y creo otras variables
# Extraer los píxeles
df2_train['Pixeles_Total'] = df2_train['ScreenResolution'].apply(extraer_pixeles)

# Extraer características de la pantalla
df2_train[['IPS', 'Touchscreen']] = df2_train['ScreenResolution'].apply(screen_featuring)

# Extraer cantidad y codificar las memorias
df2_train['Total_Memory_GB'] = df2_train['Memory'].apply(extraer_gb)
df2_train['Memory_Type_Code'] = df2_train['Memory'].apply(codificar_tipo_memoria)

# Asigno marca GPU
df2_train['GPU_Marca'] = df2_train['Gpu'].apply(asignar_marca_gpu)

# Clasificamos los CPU
df2_train['CPU_Gama'] = df2_train['Cpu'].apply(clasificar_cpu)

# Aplicar la función para extraer la velocidad de GHz
df2_train['CPU_GHz'] = df2_train['Cpu'].apply(extraer_ghz)

In [ ]:
dummies_os_com_train = pd.get_dummies(df2_train, columns=["OpSys_Grouped", "Company_Grouped"])


In [ ]:
#  Aplicar One-Hot Encoding a la variable TypeName, al ser tan pocos no hizo falta agruparlos primero
df_encoded_train = pd.get_dummies(df2_train, columns=['TypeName'])

df_encoded_train

In [ ]:
# Creo dummies para CPU_Gama

cpu_gama_dummies_train = pd.get_dummies(df2_train['CPU_Gama'], prefix='CPU')
cpu_gama_dummies_train

In [ ]:
# Creo dummies para GPU_Marca
marca_gpu_dummies_train = pd.get_dummies(df2_train['GPU_Marca'], prefix='GPU')
marca_gpu_dummies_train

In [ ]:
# Uno GPU_dummies con encoded
df_encoded_train_2 = pd.concat([df_encoded_train, marca_gpu_dummies_train], axis=1)
df_encoded_train_2

In [ ]:
# Uno CPU_dummies con Encoded
df_encoded_train_3 = pd.concat([df_encoded_train_2, cpu_gama_dummies_train], axis=1)
df_encoded_train_3

In [ ]:
# Uno CPU_dummies con Encoded
df_encoded_train_4 = pd.concat([df_encoded_train_3, dummies_os_com_train], axis=1)
df_encoded_train_4


In [ ]:
# Eliminar las columnas especificadas
df_encoded_train_4.drop(['Company', 'Product', 'Cpu', 'Memory', 'Gpu', 'OpSys'], axis=1, inplace=True)
df_encoded_train_4.drop(['OpSys_Grouped', 'Company_Grouped'], axis=1, inplace=True)
df_encoded_train_4.drop(['GPU_Marca', 'CPU_Gama', 'ScreenResolution'], axis=1, inplace=True)

In [ ]:
df_encoded_train_4.info()

In [ ]:
df_encoded_test_3.info()

In [ ]:
# m_corr = df_encoded_train_3.corr()

In [ ]:
# m_corr.to_csv('corr.csv', index=False)

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(m_corr, annot=True, fmt=".2f", annot_kws={'size': 10}, cmap='coolwarm');

## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [ ]:
# X = df_encoded_train.drop(['Price_euros'], axis=1)
# y = df_encoded_train['Price_euros']

### 2. Dividir X_train, X_test, y_train, y_test

In [ ]:
df_encoded_test_4.info()

In [ ]:
X_train = df_encoded_train_4.drop(['Price_euros'], axis=1)
X_test = df_encoded_test_4.drop(['GPU_Otro'], axis=1)
y_train = df_encoded_train_3['Price_euros']
y_test = sample['Price_euros']

In [ ]:
print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)

### 3. Crear y entrenar el modelo


In [ ]:
#creating LinearRegression Object
lm = LinearRegression()
#Training the Data Model
lm.fit(X_train, y_train)

In [ ]:
# Hago mis predicciones
predictions = lm.predict(X_test)

In [ ]:
predictions

In [ ]:
from sklearn import metrics

lm.score(X_test, y_test)

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))

In [ ]:
# from sklearn.metrics import mean_absolute_error

# # Ahora voy a escalar a ver si me da mejores resultados

# # Ahora voy a escalarlo
# from sklearn.preprocessing import StandardScaler

# # Inicializar el escalador
# scaler = StandardScaler()

# # Ajustar el escalador con los datos de entrenamiento y transformarlos
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Crear y entrenar el modelo de regresión lineal
# lm_scaled = LinearRegression()
# lm_scaled.fit(X_train_scaled, y_train)
# # Hacer predicciones con los datos de prueba escalados
# predictions_scaled = lm_scaled.predict(X_test_scaled)

# # Evaluar el modelo
# print('MAE:', mean_absolute_error(y_test, predictions_scaled))

# score_scaled = lm_scaled.score(X_test_scaled, y_test)
# print("Coeficiente de determinación (R^2) en datos escalados:", round(score_scaled), 2)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Inicializar el escalador
scaler = StandardScaler()

# Escalar solo las variables numéricas continuas, no las dummies
# Identifica las columnas que necesitas escalar
columnas_para_escalar = [ 'Ram', 'Weight/kg', 'Pixeles_Total', "Total_Memory_GB", "CPU_GHz"]

# Escalar solo esas columnas
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[columnas_para_escalar] = scaler.fit_transform(X_train[columnas_para_escalar])
X_test_scaled[columnas_para_escalar] = scaler.transform(X_test[columnas_para_escalar])

# Entrenar el modelo
lm_scaled = LinearRegression()
lm_scaled.fit(X_train_scaled, y_train)

# Evaluar el modelo
predictions_scaled = lm_scaled.predict(X_test_scaled)
print('MAE:', mean_absolute_error(y_test, predictions_scaled))
score_scaled = lm_scaled.score(X_test_scaled, y_test)
print("Coeficiente de determinación (R^2) en datos escalados:", score_scaled)


In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV

n_alphas = 100
poly = PolynomialFeatures(degree=2)
poly.fit(X_train_scaled)

X_poly_train = poly.transform(X_train_scaled)
X_poly_test = poly.transform(X_test_scaled)

pol_reg_reg_1 = Ridge()

parameters = {'alpha':np.logspace(-5, 1, n_alphas)}

gs = GridSearchCV(pol_reg_reg_1, parameters, scoring="neg_mean_absolute_error", cv=10)

gs.fit(X_poly_train, y_train)

In [ ]:
gs.best_estimator_

In [ ]:
gs.best_score_

In [ ]:
gs.best_params_

In [ ]:
poly = PolynomialFeatures(degree=2)
poly.fit(X_train_scaled)
X_poly_train = poly.transform(X_train_scaled)
X_poly_test = poly.transform(X_test_scaled)

pol_reg_reg_2 = Ridge(alpha=gs.best_params_['alpha'])
pol_reg_reg_2.fit(X_poly_train, y_train)

y_pred = pol_reg_reg_2.predict(X_poly_test)

print("MAE test", mean_absolute_error(y_test, y_pred))

In [ ]:
# # Definir el grado del polinomio
# poly = PolynomialFeatures(degree=2)
# poly.fit(X_train_scaled)

# # Transformar los datos
# X_poly_train = poly.transform(X_train_scaled)
# X_poly_test = poly.transform(X_test_scaled)

# # Definir el modelo Lasso
# lasso_reg = Lasso(max_iter=1000)

# # Definir el rango de alphas a explorar
# parameters = {'alpha': np.logspace(-4, -0.5, 100)}

# # Configurar GridSearchCV
# gs = GridSearchCV(lasso_reg, parameters, scoring="neg_mean_absolute_error", cv=10)

# # Ajustar GridSearchCV
# gs.fit(X_poly_train, y_train)

# # Mejor estimador y su score
# print("Mejor alpha:", gs.best_params_)
# print("Mejor score MAE:", -gs.best_score_)

# # Entrenar el modelo Lasso con el mejor alpha
# lasso_best = Lasso(alpha=gs.best_params_['alpha'])
# lasso_best.fit(X_poly_train, y_train)

# # Predicciones y evaluación
# y_pred_lasso = lasso_best.predict(X_poly_test)

# # Error Absoluto Medio (MAE)
# print("MAE test (Lasso)", mean_absolute_error(y_test, y_pred_lasso))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error
import numpy as np


# Definir el grado del polinomio
poly = PolynomialFeatures(degree=2)
poly.fit(X_train_scaled)

# Transformar los datos
X_poly_train = poly.transform(X_train_scaled)
X_poly_test = poly.transform(X_test_scaled)

# Definir el modelo ElasticNet
elastic_net_reg = ElasticNet(max_iter=10000)

# Definir el rango de alphas y l1_ratios a explorar
parameters = {
    'alpha': np.logspace(-4, -0.5, 100),
    'l1_ratio': np.linspace(0.01, 1, 25)
}

# Configurar GridSearchCV
gs = GridSearchCV(elastic_net_reg, parameters, scoring="neg_mean_absolute_error", cv=10)

# Ajustar GridSearchCV
gs.fit(X_poly_train, y_train)

# Mejor estimador y su score
print("Mejor alpha y l1_ratio:", gs.best_params_)
print("Mejor score MAE:", -gs.best_score_)

# Entrenar el modelo ElasticNet con los mejores parámetros
elastic_net_best = ElasticNet(alpha=gs.best_params_['alpha'], l1_ratio=gs.best_params_['l1_ratio'], max_iter=1000)
elastic_net_best.fit(X_poly_train, y_train)

# Predicciones y evaluación
y_pred_elastic_net = elastic_net_best.predict(X_poly_test)

# Error Absoluto Medio (MAE)
print("MAE test (ElasticNet)", mean_absolute_error(y_test, y_pred_elastic_net))


In [ ]:
dicccionario ={'id':df_encoded_test.index,"Price_euros": y_pred}

In [ ]:
df_predi = pd.DataFrame(dicccionario)
df_predi

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [ ]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                df_to_submit.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [ ]:
df_predi.to_csv('poly_2.csv', index=False)